In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain. prompts import PromptTemplate



In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# create pinecone cluster (we use pincone vector db)
#generate api key from pinecone website


In [ ]:

#PINECONE_API_ENV = "gcp-starter"

In [ ]:
#Extract data from the PDF
#only when we have more than one .pdf 

#from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
#def load_pdf(data):
#    loader = DirectoryLoader(data,
#                    glob="*.pdf",
#                    loader_cls=PyPDFLoader)
    
#    documents = loader.load()

#    return documents
#extracted_data = load_pdf(r"C:\Users\Sniper\AI Project\Chatbot-with-Llma-2\data\Medical_book.pdf")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
#Extract data from the PDF
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

extracted_data = load_pdf(r"C:\Users\Sniper\AI Project\Chatbot-with-Llma-2\data\Medical_book.pdf")

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [ ]:
# text_chunks


In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
# query_result

In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])